# FLPA Simulation with Defense Mechanism
This notebook simulates a federated learning poisoning attack scenario with a basic defense mechanism that filters out low-accuracy clients.

In [ ]:
# 📦 Step 1: Upload CSV files (client_1_clean.csv, client_2_clean.csv, client_3_poisoned.csv)
from google.colab import files
import pandas as pd
uploaded = files.upload()

In [ ]:
# 📊 Step 2: Read client datasets
client_data = {name: pd.read_csv(name) for name in uploaded}
for name, df in client_data.items():
    print(f"{name} shape:", df.shape)

In [ ]:
# ⚙️ Step 3: Define training and evaluation function
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def train_and_evaluate(df):
    X = df.drop("Label", axis=1)
    y = df["Label"]
    split = int(0.8 * len(df))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]
    model = RandomForestClassifier(n_estimators=100)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return model, acc

In [ ]:
# 🛡️ Step 4: Simulate federated learning with defense (accuracy threshold)
client_models = []
client_weights = []
threshold = 0.6  # clients below this accuracy will be excluded

for name, df in client_data.items():
    model, acc = train_and_evaluate(df)
    print(f"{name} accuracy: {acc:.2f}")
    if acc >= threshold:
        client_models.append(model)
        client_weights.append(len(df))
    else:
        print(f"⚠️ {name} excluded due to low accuracy.")

In [ ]:
# 📈 Step 5: Train global model on combined clean data
if client_models:
    # Merge clean data
    combined = pd.concat([client_data[name] for name, df in client_data.items() if name in uploaded and train_and_evaluate(df)[1] >= threshold])
    global_model, global_acc = train_and_evaluate(combined)
    print(f"\n🌐 Global model accuracy after defense: {global_acc:.2f}")
else:
    print("No valid clients available to train global model.")